# Run contagion simulations
Simulate the spread of information on networks.

In [1]:
# Add project root to Python path
import sys
from pathlib import Path

project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# Import necessary libraryes
import os
import joblib
import numpy as np
import pandas as pd
import plotnine as p9
import psutil
from numpy import random

from src.population_density_networks import model
from src.population_density_networks import contagion

## Modeling Parameters

In [2]:
# ----- GENERAL -----
# Save paths
OUTPUT_DIR = '../data_derived'

# Simulation
N_JOBS = psutil.cpu_count(logical=True) - 1


# ----- NETWORK MODEL -----
# Fixed global parameters
N = 1000
K_CAP_MEAN = 50
K_CAP_SD = 0.5 * K_CAP_MEAN
RADIUS = 1.0

# Varying parameters
# DENSITIES = [1e-4, 1e4]
DENSITIES = np.logspace(-5, 5, 21, base=10)

# Simulation parameters
NETWORK_SIMULATION_ROUNDS = 2 * N * K_CAP_MEAN
NETWORK_SIMULATION_REPLICATES = 50


# ----- CONTAGION MODEL -----
# Simple contagion model parameters
SIMPLE_INITIAL_INFECTED = 1
SIMPLE_INFECTION_PROB = 0.01

# Complex contagion model parameters
COMPLEX_INITIAL_INFECTED = 0.05

# Simulation parameters
CONATGION_SIMULATION_ROUNDS = 100
CONTAGION_SIMULATION_REPLICATES = 50  # per simulated network



## Run Simulations Across Population Density

In [4]:
#####################
# Function to run full network + contagion model
#####################
# Simulation code
def run_full_contagion_simulation(
    # Network model parameters
    density: float,
    replicate: int,
    n: int,
    k_cap_mean: float,
    k_cap_sd: float,
    radius: float, 
    network_simulation_rounds: int,
    # Contagion model parameters
    simple_initial_infected: int,
    simple_infection_probability: float,
    complex_initial_infected: float,
    contagion_simulation_rounds: int,
    contagion_simulation_replicates: int,
) -> pd.DataFrame:
    # Run full network model to generate network
    network_siumlation = model.run_single_simulation(
        density=density,
        replicate=replicate,
        n=n, 
        k_cap_mean=k_cap_mean, 
        k_cap_sd=k_cap_sd,
        radius=radius,
        simulation_rounds=network_simulation_rounds,
        return_network=True,
    )
    network = network_siumlation['social_network']

    # Run contagion simulations
    simple_contagion_results = []
    complex_contagion_results = []
    for _ in range(contagion_simulation_replicates):

        # Simple contagion model
        simple_contagion_model = contagion.SimpleContagionModel(
            network=network,
            initial_infected=simple_initial_infected,
            infection_probability=simple_infection_probability,
        )
        simple_contagion_model.run_simulation(time_steps=contagion_simulation_rounds)

        # Complex contagion model
        thresholds = random.uniform(
            low=0.0,
            high=1.0,
            size=network.shape[0]
        )
        complex_contagion_model = contagion.ComplexContagionModel(
            network=network,
            initial_infected=complex_initial_infected,
            thresholds=thresholds
        )
        complex_contagion_model.run_simulation(time_steps=contagion_simulation_rounds)

        # Analyze results
        simple_contagion_results.append(contagion.analyze_contagion_results(simple_contagion_model))
        complex_contagion_results.append(contagion.analyze_contagion_results(complex_contagion_model))

    # Create full simple contagion results
    simple_contagion_results = pd.DataFrame(simple_contagion_results)
    simple_contagion_results.insert(0, 'contagion_type', 'simple')
    simple_contagion_results.insert(1, 'contagion_replicate', np.arange(CONTAGION_SIMULATION_REPLICATES))

    # Create full complex contagion results
    complex_contagion_results = pd.DataFrame(complex_contagion_results)
    complex_contagion_results.insert(0, 'contagion_type', 'complex')
    complex_contagion_results.insert(1, 'contagion_replicate', np.arange(CONTAGION_SIMULATION_REPLICATES))


    # Combine results
    contagion_results = pd.concat([simple_contagion_results, complex_contagion_results], ignore_index=True)
    contagion_results.insert(1, 'population_density', density)
    contagion_results.insert(2, 'k_cap_mean', k_cap_mean)
    contagion_results.insert(3, 'k_cap_sd', k_cap_sd)
    contagion_results.insert(4, 'network_replicate', replicate)
    
    return contagion_results

In [10]:
#####################
# Run simulations
#####################
# Generate all parameter combinations
param_combinations = [
    (
        density,
        replicate, 
        N,
        K_CAP_MEAN,
        K_CAP_SD,
        RADIUS,
        NETWORK_SIMULATION_ROUNDS,
        SIMPLE_INITIAL_INFECTED,
        SIMPLE_INFECTION_PROB,
        COMPLEX_INITIAL_INFECTED,
        CONATGION_SIMULATION_ROUNDS,
        CONTAGION_SIMULATION_REPLICATES,
    )
    for density in DENSITIES
    for replicate in range(NETWORK_SIMULATION_REPLICATES)
]

print(f"🚀 Running {len(param_combinations)} simulations in parallel...")

# Run simulations in parallel
parallel_jobs = min(N_JOBS, len(param_combinations))
results_list = joblib.Parallel(n_jobs=parallel_jobs, verbose=10)(
    joblib.delayed(run_full_contagion_simulation)(*params) for params in param_combinations
)

# Convert results to DataFrame
results = pd.concat(results_list, ignore_index=True)

🚀 Running 1050 simulations in parallel...


[Parallel(n_jobs=13)]: Using backend LokyBackend with 13 concurrent workers.
[Parallel(n_jobs=13)]: Done   6 tasks      | elapsed:   52.6s
[Parallel(n_jobs=13)]: Done  15 tasks      | elapsed:  1.8min
[Parallel(n_jobs=13)]: Done  24 tasks      | elapsed:  1.9min
[Parallel(n_jobs=13)]: Done  35 tasks      | elapsed:  2.8min
[Parallel(n_jobs=13)]: Done  46 tasks      | elapsed:  3.7min
[Parallel(n_jobs=13)]: Done  59 tasks      | elapsed:  4.6min
[Parallel(n_jobs=13)]: Done  72 tasks      | elapsed:  5.5min
[Parallel(n_jobs=13)]: Done  87 tasks      | elapsed:  6.4min
[Parallel(n_jobs=13)]: Done 102 tasks      | elapsed:  7.5min
[Parallel(n_jobs=13)]: Done 119 tasks      | elapsed:  9.1min
[Parallel(n_jobs=13)]: Done 136 tasks      | elapsed: 10.0min
[Parallel(n_jobs=13)]: Done 155 tasks      | elapsed: 11.1min
[Parallel(n_jobs=13)]: Done 174 tasks      | elapsed: 12.7min
[Parallel(n_jobs=13)]: Done 195 tasks      | elapsed: 13.8min
[Parallel(n_jobs=13)]: Done 216 tasks      | elapsed: 1

In [11]:
results

,contagion_type,population_density,k_cap_mean,k_cap_sd,network_replicate,contagion_replicate,final_infected_fraction,time_to_majority,reached_majority_spread,max_slope
0,simple,0.00001,50,25.0,0,0,1.000,25.0,True,0.099
1,simple,0.00001,50,25.0,0,1,1.000,32.0,True,0.085
2,simple,0.00001,50,25.0,0,2,1.000,19.0,True,0.094
3,simple,0.00001,50,25.0,0,3,1.000,39.0,True,0.090
4,simple,0.00001,50,25.0,0,4,1.000,25.0,True,0.103
...,...,...,...,...,...,...,...,...,...,...
104995,complex,100000.00000,50,25.0,49,45,0.629,14.0,True,0.044
104996,complex,100000.00000,50,25.0,49,46,0.697,8.0,True,0.070
104997,complex,100000.00000,50,25.0,49,47,0.235,NaN,False,0.033
104998,complex,100000.00000,50,25.0,49,48,0.456,NaN,False,0.060


## Save results

In [12]:
#####################
# Save data
#####################
# Break into separate dataframes for each contagion type
simple_results = results[results['contagion_type'] == 'simple']
complex_results = results[results['contagion_type'] == 'complex']
        
# # Write to file
simple_results.to_csv(f'{OUTPUT_DIR}/contagion_simple_results.csv', index=False)
complex_results.to_csv(f'{OUTPUT_DIR}/contagion_complex_results.csv', index=False)

## Run Example Simulations for Time Series
Here, we'll simulate a few simulations at a low and high population density so we can examine the fine-scale timeseries more closely.

In [3]:
#####################
# Function to run full network + contagion model and extract timeseries of infection
#####################
# Simulation code
def generate_contagion_simulation_timeseries(
    # Network model parameters
    density: float,
    replicate: int,
    n: int,
    k_cap_mean: float,
    k_cap_sd: float,
    radius: float, 
    network_simulation_rounds: int,
    # Contagion model parameters
    simple_initial_infected: int,
    simple_infection_probability: float,
    complex_initial_infected: float,
    contagion_simulation_rounds: int,
    contagion_simulation_replicates: int,
) -> pd.DataFrame:
    # Run full network model to generate network
    network_siumlation = model.run_single_simulation(
        density=density,
        replicate=replicate,
        n=n, 
        k_cap_mean=k_cap_mean, 
        k_cap_sd=k_cap_sd,
        radius=radius,
        simulation_rounds=network_simulation_rounds,
        return_network=True,
    )
    network = network_siumlation['social_network']

    # Run contagion simulations
    simple_contagion_timeseries = pd.DataFrame([])
    complex_contagion_timeseries = pd.DataFrame([])
    for i in range(contagion_simulation_replicates):

        # Simple contagion model
        simple_contagion_model = contagion.SimpleContagionModel(
            network=network,
            initial_infected=simple_initial_infected,
            infection_probability=simple_infection_probability,
        )
        simple_contagion_model.run_simulation(time_steps=contagion_simulation_rounds)

        # Complex contagion model
        thresholds = random.uniform(
            low=0.0,
            high=1.0,
            size=network.shape[0]
        )
        complex_contagion_model = contagion.ComplexContagionModel(
            network=network,
            initial_infected=complex_initial_infected,
            thresholds=thresholds
        )
        complex_contagion_model.run_simulation(time_steps=contagion_simulation_rounds)

        # Extract timeseries of infection
        simple_contagion_model.state_time_series.insert(0, 'contagion_replicate', i)
        simple_contagion_timeseries = pd.concat([simple_contagion_timeseries, simple_contagion_model.state_time_series], ignore_index=True)

        complex_contagion_model.state_time_series.insert(0, 'contagion_replicate', i)
        complex_contagion_timeseries = pd.concat([complex_contagion_timeseries, complex_contagion_model.state_time_series], ignore_index=True)

    # Combine results
    simple_contagion_timeseries.insert(0, 'contagion_type', 'simple')
    complex_contagion_timeseries.insert(0, 'contagion_type', 'complex')
    timeseries_data = pd.concat([simple_contagion_timeseries, complex_contagion_timeseries], ignore_index=True)
    timeseries_data.insert(1, 'population_density', density)
    timeseries_data.insert(2, 'k_cap_mean', k_cap_mean)
    timeseries_data.insert(3, 'k_cap_sd', k_cap_sd)
    timeseries_data.insert(4, 'network_replicate', replicate)
    return timeseries_data

In [8]:
#####################
# Run simulations
#####################
# Set example parameters
EXAMPLE_DENSITIES = [1e-4, 1e4]
EXAMPLE_NETWORK_REPLICATES = 50

# Generate all parameter combinations
param_combinations = [
    (
        density,
        replicate, 
        N,
        K_CAP_MEAN,
        K_CAP_SD,
        RADIUS,
        NETWORK_SIMULATION_ROUNDS,
        SIMPLE_INITIAL_INFECTED,
        SIMPLE_INFECTION_PROB,
        COMPLEX_INITIAL_INFECTED,
        CONATGION_SIMULATION_ROUNDS,
        CONTAGION_SIMULATION_REPLICATES,
    )
    for density in EXAMPLE_DENSITIES
    for replicate in range(EXAMPLE_NETWORK_REPLICATES)
]

print(f"🚀 Running {len(param_combinations)} simulations in parallel...")

# Run simulations in parallel
parallel_jobs = min(N_JOBS, len(param_combinations))
results_list = joblib.Parallel(n_jobs=parallel_jobs, verbose=10)(
    joblib.delayed(generate_contagion_simulation_timeseries)(*params) for params in param_combinations
)

# Convert results to DataFrame
timeseries_results = pd.concat(results_list, ignore_index=True)

🚀 Running 100 simulations in parallel...


[Parallel(n_jobs=13)]: Using backend LokyBackend with 13 concurrent workers.
[Parallel(n_jobs=13)]: Done   6 tasks      | elapsed:   53.0s
[Parallel(n_jobs=13)]: Done  15 tasks      | elapsed:  1.7min
[Parallel(n_jobs=13)]: Done  24 tasks      | elapsed:  2.0min
[Parallel(n_jobs=13)]: Done  35 tasks      | elapsed:  2.6min
[Parallel(n_jobs=13)]: Done  46 tasks      | elapsed:  3.6min
[Parallel(n_jobs=13)]: Done  59 tasks      | elapsed:  4.3min
[Parallel(n_jobs=13)]: Done  72 tasks      | elapsed:  5.0min
[Parallel(n_jobs=13)]: Done  86 out of 100 | elapsed:  5.7min remaining:   55.6s
[Parallel(n_jobs=13)]: Done  97 out of 100 | elapsed:  6.2min remaining:   11.6s
[Parallel(n_jobs=13)]: Done 100 out of 100 | elapsed:  6.3min finished


In [10]:
#####################
# Save timeseries data
#####################
# Separate into two dataframes for each contagion type to save memory
simple_timeseries = timeseries_results[timeseries_results['contagion_type'] == 'simple']
complex_timeseries = timeseries_results[timeseries_results['contagion_type'] == 'complex']

# Save to file
simple_timeseries.to_csv(f'{OUTPUT_DIR}/contagion_timeseries_simple.csv', index=False)
complex_timeseries.to_csv(f'{OUTPUT_DIR}/contagion_timeseries_complex.csv', index=False)